In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
from mysql.connector import Error
import csv

connection = None

def fetch_tables():
    try:
        connection = mysql.connector.connect(
            host='localhost',  # MySQL server host
            user='root',  # MySQL username
            password='Devyani',  # MySQL password
            database='mng'  # database name
        )

        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()

            table_listbox.delete(0, tk.END)  # Clear the listbox
            for table in tables:
                table_listbox.insert(tk.END, table[0])
            
            cursor.close()
            connection.close()

    except Error as e:
        messagebox.showerror("Error", f"Error while connecting to MySQL: {e}")

# Function to fetch data from the selected table
def fetch_table_data():
    try:
        selected_table = table_listbox.get(tk.ACTIVE)
        if not selected_table:
            messagebox.showwarning("Warning", "No table selected")
            return

        connection = mysql.connector.connect(
            host='localhost',  
            user='root',  
            password='Devyani',  
            database='mng'  
        )

        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(f"SELECT * FROM {selected_table}")
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()

            for col in data_tree.get_children():
                data_tree.delete(col)
            
            data_tree["columns"] = columns
            data_tree["show"] = "headings"
            for col in columns:
                data_tree.heading(col, text=col)
                data_tree.column(col, width=100)

            for row in rows:
                data_tree.insert("", "end", values=row)
            
            cursor.close()
            connection.close()

    except Error as e:
        messagebox.showerror("Error", f"Error while connecting to MySQL: {e}")

# Function to fetch specific data based on a query
def fetch_query_data():
    query = query_entry.get()
    if not query:
        messagebox.showwarning("Warning", "Query cannot be empty")
        return

    try:
        connection = mysql.connector.connect(
            host='localhost',  
            user='root',  
            password='Devyani',  
            database='mng'  
        )

        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()

            for col in data_tree.get_children():
                data_tree.delete(col)
            
            data_tree["columns"] = columns
            data_tree["show"] = "headings"
            for col in columns:
                data_tree.heading(col, text=col)
                data_tree.column(col, width=100)

            for row in rows:
                data_tree.insert("", "end", values=row)
            
            cursor.close()
            connection.close()

    except Error as e:
        messagebox.showerror("Error", f"Error while executing query: {e}")


root = tk.Tk()
root.title("MySQL Database Information")

frame = tk.Frame(root)
frame.pack(pady=20, padx=20)

btn_fetch_tables = tk.Button(frame, text="Fetch Tables", command=fetch_tables)
btn_fetch_tables.grid(row=0, column=0, pady=10)

table_listbox = tk.Listbox(frame, height=10)
table_listbox.grid(row=1, column=0, pady=10)

btn_fetch_table_data = tk.Button(frame, text="Fetch Table Data", command=fetch_table_data)
btn_fetch_table_data.grid(row=2, column=0, pady=10)

query_label = tk.Label(frame, text="Enter Query:")
query_label.grid(row=3, column=0, pady=5)
query_entry = tk.Entry(frame, width=50)
query_entry.grid(row=4, column=0, pady=5)

btn_fetch_query_data = tk.Button(frame, text="Execute Query", command=fetch_query_data)
btn_fetch_query_data.grid(row=5, column=0, pady=10)

data_tree = ttk.Treeview(root)
data_tree.pack(pady=20)

root.mainloop()

